In [1]:
import pandas as pd
import datetime
from ratelimit import limits, sleep_and_retry
from keys import *

In [2]:
data = pd.read_csv('tweet_ids.utf-8.csv')

all_ids = list(data['id'])

In [3]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")


tweets_list = []


def create_url(ids):
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
#     ids = "ids=1278747501642657792,1255542774432063488"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

@sleep_and_retry
@limits(calls=1, period=3)
def main(ids):
    url = create_url(ids)
    json_response = connect_to_endpoint(url)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    tweets_list.append(json_response['data'])


# if __name__ == "__main__":
#     main()

In [4]:
def run(no_of_tweets):
    
    batches = no_of_tweets // 100
    
    for i in range(batches): 
        
        tweet_ids = all_ids[i*100: i*100+100]
        
        ids = 'ids=' + str(tweet_ids[0])
        for j in range(99):
            ids = ids + ',' + str(tweet_ids[j+1])
        
        main(ids)
        
        print (f'get_tweets was run {i+1} times!\n')

In [5]:
begin_time = datetime.datetime.now()

run(1000)

end_time = datetime.datetime.now()
runtime = end_time - begin_time
print (f'runtime: {runtime}')

get_tweets was run 1 times!

get_tweets was run 2 times!

get_tweets was run 3 times!

get_tweets was run 4 times!

get_tweets was run 5 times!

get_tweets was run 6 times!

get_tweets was run 7 times!

get_tweets was run 8 times!

get_tweets was run 9 times!

get_tweets was run 10 times!

runtime: 0:00:25.656447


In [10]:
tweets_list[0][0]

{'edit_history_tweet_ids': ['1120477524633505792'],
 'lang': 'es',
 'id': '1120477524633505792',
 'text': 'RT @MayoralRafa: 60.000 millones, 50.000 millones, 100.000 millones y hasta 200.000 millones de rescate bancario, todo cifras oficiales. Ci…',
 'author_id': '266637594'}